# Utilization of DenseNet201 for diagnosis of breast abnormality

**Abstract:** As one of the leading killers of females, breast cancer has become one of the heated research topics in the community of clinical medical science and computer science. In the clinic, mammography is a publicly accepted technique to detect early abnormalities such as masses and distortions in breast leading to cancer. Interpreting the images, however, is time-consuming and error-prone for radiologists considering artificial factors including potential fatigue. To improve radiologists’ working efficiency, we developed a semi-automatic computer-aided diagnosis system to classify mammograms into normality and abnormality and thus to ease the process of making a diagnosis of breast cancer. Through transferring deep convolutional neural network DenseNet201 on the basis of suspicious regions provided by radiologists into our system, we obtained the network we termed DenseNet201-C, which achieved a high diagnostic accuracy of 92.73%. The comparison results between our method and the other five methods show that our method achieved the highest accuracy.

In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import pydicom
import imageio
import shutil
import tensorflow as tf
import numpy as np
import cv2

from PIL import Image

2025-07-20 11:55:39.733307: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-20 11:55:39.733499: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-20 11:55:39.891072: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
def pipeline(image, label):
    if tf.executing_eagerly():
      opened_image = opening(image.numpy(), element)   # Apply morphological Openess
      mask = create_binarized_mask(opened_image) # Create the Binary Mask
      image = opened_image * mask
    return image, label

In [5]:
def scheduler(epoch, lr):
     if epoch % 10 == 0:
         return lr - 0.01
     else:
         return lr

In [6]:
def evaluate(model, validation):
  m = tf.keras.metrics.Accuracy()
  for image, label in validation:
      pred = model.predict(image)
      pred = tf.constant(pred > 0.5, dtype=tf.int64)

      m.update_state(pred, tf.expand_dims(label, 1))

  return m.result()

In [7]:
def init_model(freeze=False):
  pretrained_model = tf.keras.applications.densenet.DenseNet201(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=None,
        pooling=None,
        classes=None,
        classifier_activation=None
    )
  if freeze:
    for layer in pretrained_model.layers:
        layer.trainable = False

  last_layer = pretrained_model.get_layer('conv5_block32_concat')
  x = tf.keras.layers.GlobalAveragePooling2D()(last_layer.output)
  x =  tf.keras.layers.Flatten()(x)
  x =  tf.keras.layers.Dropout(0.5)(x)
  out = tf.keras.layers.Dense(1, activation='sigmoid')(x)
  out

  return tf.keras.Model(pretrained_model.input, out)

# Converting to PNG and saving locally


"Creating a temporary directory where the images in PNG format will be stored

In [8]:
final_dir = os.path.join('/kaggle/working/AllPNGs')
os.makedirs(final_dir, exist_ok=True)

Converting from .dcm to .png, and saving the images in the new directory.

In [9]:
directory = "/kaggle/input/inbreast-dataset/INbreast Release 1.0/AllDICOMs/"

for filename in os.listdir(directory):
    if filename.endswith(".dcm"): 
        ds = pydicom.dcmread(os.path.join(directory, filename))
        pixel_array = ds.pixel_array
        name_parts = filename.split("_")        
        new_filename = name_parts[0]
        output_path = f"/kaggle/working/AllPNGs/{new_filename}.png"
        imageio.imwrite(output_path, pixel_array)

# Creating the class split and classes

In [10]:
os.listdir('/kaggle/input/inbreast-dataset/INbreast Release 1.0')

['INbreast.csv',
 'README.txt',
 'inbreast.pdf',
 'PectoralMuscle',
 'INbreast.xls',
 'AllXML',
 'AllDICOMs',
 'AllROI',
 'MedicalReports']

In [11]:
raw_csv = pd.read_csv("/kaggle/input/inbreast-dataset/INbreast Release 1.0/INbreast.csv", delimiter=';')
raw_csv = raw_csv.rename(columns={'File Name':'FileName'})

In [12]:
raw_csv_ = pd.read_csv("/kaggle/input/inbreast-roi-yolov8l/INBREAST-Mammography/description.csv")
raw_csv_ = raw_csv_.rename(columns={'File name':'FileName'})

In [13]:
raw_csv.head()

,Patient ID,Patient age,Laterality,View,Acquisition date,FileName,ACR,Bi-Rads
0,removed,removed,R,CC,201001,22678622,4,1
1,removed,removed,L,CC,201001,22678646,4,3
2,removed,removed,R,MLO,201001,22678670,4,1
3,removed,removed,L,MLO,201001,22678694,4,3
4,removed,removed,R,CC,201001,22614074,2,5


In [14]:
raw_csv['View'].value_counts()

View
MLO    206
CC     203
FB       1
Name: count, dtype: int64

# Create a new dataframe thst will contain CC views 

In [15]:
# Filter rows where the View is 'CC'
CC_Views = raw_csv[raw_csv['View'] == 'CC']

# Optionally preview the filtered results
print(CC_Views.head())


   Patient ID Patient age Laterality View  Acquisition date  FileName ACR  \
0     removed     removed          R   CC            201001  22678622   4   
1     removed     removed          L   CC            201001  22678646   4   
4     removed     removed          R   CC            201001  22614074   2   
5     removed     removed          L   CC            201001  22614097   2   
10    removed     removed          L   CC            201001  50997488   3   

   Bi-Rads  
0        1  
1        3  
4        5  
5        2  
10       2  


In [16]:
CC_Views.value_counts()

Patient ID  Patient age  Laterality  View  Acquisition date  FileName  ACR  Bi-Rads
removed     removed      L           CC    200801            24055483  1    4c         1
                         R           CC    200902            20588164  2    1          1
                                           200901            51070197  2    2          1
                                                             53582818  2    2          1
                                           200902            20586908  2    2          1
                                                                                      ..
                         L           CC    201001            50993841  3    2          1
                                                             50994787  3    1          1
                                                             50996110  1    2          1
                                                             50996228  4    2          1
                         R

In [27]:
print(CC_Views['Bi-Rads'].value_counts())

Bi-Rads
2     109
1      33
5      24
3      11
4c     11
4a      7
6       4
4b      4
Name: count, dtype: int64


In [28]:
# Map BI-RADS to binary labels (benign: 1,2,3; malignant: 4,5,6)
CC_Views = CC_Views.copy()


Label
1    203
Name: count, dtype: int64

In [31]:
# Clean Bi-Rads column: convert strings like '4a', '4b', '4c' to numeric 4
def clean_birads(value):
    if isinstance(value, str):
        # Extract the numeric part (e.g., '4a' -> '4', '4c' -> '4')
        value = ''.join(filter(str.isdigit, value))
    try:
        return int(value)
    except (ValueError, TypeError):
        return np.nan  # Handle invalid entries as NaN

In [32]:
CC_Views['Bi-Rads_Cleaned'] = CC_Views['Bi-Rads'].apply(clean_birads)

In [33]:

# Check for NaN values after cleaning
print("NaN values in Bi-Rads_Cleaned:", CC_Views['Bi-Rads_Cleaned'].isna().sum())

NaN values in Bi-Rads_Cleaned: 0


In [34]:

# Map BI-RADS to binary labels (benign: 1,2,3; malignant: 4,5,6)
CC_Views['Label'] = CC_Views['Bi-Rads_Cleaned'].apply(lambda x: 0 if x in [1, 2, 3] else 1)


In [36]:
CC_Views.head()

,Patient ID,Patient age,Laterality,View,Acquisition date,FileName,ACR,Bi-Rads,Label,Bi-Rads_Cleaned
0,removed,removed,R,CC,201001,22678622.png.png,4,1,0,1
1,removed,removed,L,CC,201001,22678646.png.png,4,3,0,3
4,removed,removed,R,CC,201001,22614074.png.png,2,5,1,5
5,removed,removed,L,CC,201001,22614097.png.png,2,2,0,2
10,removed,removed,L,CC,201001,50997488.png.png,3,2,0,2


In [37]:
print("Sample of cleaned data:\n", CC_Views[['FileName', 'Bi-Rads', 'Bi-Rads_Cleaned', 'Label']].head())

Sample of cleaned data:
             FileName Bi-Rads  Bi-Rads_Cleaned  Label
0   22678622.png.png       1                1      0
1   22678646.png.png       3                3      0
4   22614074.png.png       5                5      1
5   22614097.png.png       2                2      0
10  50997488.png.png       2                2      0


In [41]:
CC_Views['Label'].value_counts()

Label
0    153
1     50
Name: count, dtype: int64

In [ ]:
 
# Prepare image paths and labels
CC_Views['FileName'] = CC_Views['FileName'].astype(str) + ".png"
image_paths = [os.path.join(output_dir, fname) for fname in CC_Views['FileName']]
labels = CC_Views['Label'].values

In [ ]:
# Step 1: Split dataset into train (70%), validation (20%), and test (10%)
# First split: 70% train, 30% temp (val + test)
train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    image_paths, labels, test_size=0.3, random_state=42, stratify=labels
)
# Second split: 20% validation (2/3 of temp), 10% test (1/3 of temp)
val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=0.333, random_state=42, stratify=temp_labels
)

In [ ]:
# Verify split sizes
print(f"Train set: {len(train_paths)} samples")
print(f"Validation set: {len(val_paths)} samples")
print(f"Test set: {len(test_paths)} samples")

In [ ]:
# Step 2: Balance the training set using SMOTE
# SMOTE requires numerical features, so we use indices and resample paths/labels
# Explanation: The original training set is imbalanced (approx. 107 benign, 35 malignant).
# SMOTE generates synthetic samples for the minority class (malignant) by interpolating
# between existing samples, balancing the classes to have equal counts (approx. 107 each).
train_indices = np.arange(len(train_paths)).reshape(-1, 1)
smote = SMOTE(random_state=42)
train_indices_resampled, train_labels_resampled = smote.fit_resample(train_indices, train_labels)
# Map resampled indices back to paths
train_paths_resampled = [train_paths[i[0]] for i in train_indices_resampled]

In [ ]:
# Verify balanced training set
print("Balanced training label distribution:\n", pd.Series(train_labels_resampled).value_counts())

In [ ]:
# Morphological opening and mask creation
def opening(image, element=np.ones((5, 5))):
    return morphology.grey_opening(image, structure=element)

def create_binarized_mask(image):
    _, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary / 255.0

# Image preprocessing pipeline
def pipeline(image, label):
    if tf.executing_eagerly():
        opened_image = opening(image.numpy(), np.ones((5, 5)))
        mask = create_binarized_mask(opened_image)
        image = opened_image * mask
    return image, label

def tf_pipeline(image, label):
    [image, label] = tf.py_function(pipeline, [image, label], [tf.float32, tf.int32])
    image.set_shape([None, None, 3])
    label.set_shape([])
    return image, label

In [ ]:
Load and preprocess images
def load_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image, label

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_paths_resampled, train_labels_resampled))
val_dataset = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))

train_dataset = (train_dataset
                 .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
                 .map(tf_pipeline, num_parallel_calls=tf.data.AUTOTUNE)
                 .batch(16)
                 .prefetch(tf.data.AUTOTUNE))

val_dataset = (val_dataset
               .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
               .map(tf_pipeline, num_parallel_calls=tf.data.AUTOTUNE)
               .batch(16)
               .prefetch(tf.data.AUTOTUNE))

test_dataset = (test_dataset
                .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
                .map(tf_pipeline, num_parallel_calls=tf.data.AUTOTUNE)
                .batch(16)
                .prefetch(tf.data.AUTOTUNE))

In [ ]:
# Learning rate scheduler
def scheduler(epoch, lr):
    if epoch % 10 == 0:
        return max(lr - 0.01, 0.0001)
    return lr

# Initialize DenseNet201 model
def init_model(freeze=False):
    pretrained_model = tf.keras.applications.densenet.DenseNet201(
        include_top=False,
        weights='imagenet',
        input_shape=(224, 224, 3)
    )
    if freeze:
        for layer in pretrained_model.layers:
            layer.trainable = False
    last_layer = pretrained_model.get_layer('conv5_block32_concat')
    x = tf.keras.layers.GlobalAveragePooling2D()(last_layer.output)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    out = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    return tf.keras.Model(pretrained_model.input, out)

# Evaluation function
def evaluate(model, dataset):
    m = tf.keras.metrics.BinaryAccuracy()
    for image, label in dataset:
        pred = model.predict(image)
        pred = tf.constant(pred > 0.5, dtype=tf.float32)
        m.update_state(pred, tf.cast(tf.expand_dims(label, 1), tf.float32))
    return m.result().numpy()

# Initialize and compile model
model = init_model(freeze=True)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)

# Learning rate callback
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Train model
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[lr_callback]
)

# Evaluate on validation and test sets
val_accuracy = evaluate(model, val_dataset)
test_accuracy = evaluate(model, test_dataset)
print(f"Validation Binary Accuracy: {val_accuracy:.4f}")
print(f"Test Binary Accuracy: {test_accuracy:.4f}")

# Save model
model.save('/kaggle/working/cc_view_classifier_balanced.h5')

In [24]:
from sklearn.model_selection import train_test_split


# Split dataset
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42, stratify=labels
)

# Morphological opening and mask creation
def opening(image, element=np.ones((5, 5))):
    return morphology.grey_opening(image, structure=element)

def create_binarized_mask(image):
    _, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary / 255.0

# Image preprocessing pipeline
def pipeline(image, label):
    if tf.executing_eagerly():
        opened_image = opening(image.numpy(), np.ones((5, 5)))
        mask = create_binarized_mask(opened_image)
        image = opened_image * mask
    return image, label

# Wrap pipeline for TensorFlow
def tf_pipeline(image, label):
    [image, label] = tf.py_function(pipeline, [image, label], [tf.float32, tf.int32])
    image.set_shape([None, None, 3])
    label.set_shape([])
    return image, label

# Load and preprocess images
def load_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0  # Normalize to [0,1]
    return image, label

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))

train_dataset = (train_dataset
                 .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
                 .map(tf_pipeline, num_parallel_calls=tf.data.AUTOTUNE)
                 .batch(16)
                 .prefetch(tf.data.AUTOTUNE))

val_dataset = (val_dataset
               .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
               .map(tf_pipeline, num_parallel_calls=tf.data.AUTOTUNE)
               .batch(16)
               .prefetch(tf.data.AUTOTUNE))

# Learning rate scheduler
def scheduler(epoch, lr):
    if epoch % 10 == 0:
        return max(lr - 0.01, 0.0001)
    return lr

# Initialize model
def init_model(freeze=False):
    pretrained_model = tf.keras.applications.densenet.DenseNet201(
        include_top=False,
        weights='imagenet',
        input_shape=(224, 224, 3)
    )
    if freeze:
        for layer in pretrained_model.layers:
            layer.trainable = False
    last_layer = pretrained_model.get_layer('conv5_block32_concat')
    x = tf.keras.layers.GlobalAveragePooling2D()(last_layer.output)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    out = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    return tf.keras.Model(pretrained_model.input, out)

# Evaluation function
def evaluate(model, validation):
    m = tf.keras.metrics.BinaryAccuracy()
    for image, label in validation:
        pred = model.predict(image)
        pred = tf.constant(pred > 0.5, dtype=tf.float32)
        m.update_state(pred, tf.cast(tf.expand_dims(label, 1), tf.float32))
    return m.result().numpy()

# Initialize and compile model
model = init_model(freeze=True)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)

# Learning rate callback
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Train model
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[lr_callback]
)

# Evaluate model
accuracy = evaluate(model, val_dataset)
print(f"Validation Binary Accuracy: {accuracy:.4f}")

# Save model
model.save('/kaggle/working/cc_view_classifier.h5')

74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/30


NotFoundError: Graph execution error:

Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
/kaggle/working/AllPNGs/53587663.png.png; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]] [Op:__inference_one_step_on_iterator_35177]

# Create a new df with MLO views Images 

In [17]:
# Filter rows where the View is 'CC'
MLO_Views = raw_csv[raw_csv['View'] == 'MLO']

# Optionally preview the filtered results
print(MLO_Views.head())


  Patient ID Patient age Laterality View  Acquisition date  FileName ACR  \
2    removed     removed          R  MLO            201001  22678670   4   
3    removed     removed          L  MLO            201001  22678694   4   
6    removed     removed          R  MLO            201001  22614127   2   
7    removed     removed          L  MLO            201001  22614150   2   
8    removed     removed          L  MLO            201001  50997434   3   

  Bi-Rads  
2       1  
3       3  
6       5  
7       2  
8       2  


In [22]:
MLO_Views.value_counts()

Patient ID  Patient age  Laterality  View  Acquisition date  FileName  ACR  Bi-Rads
removed     removed      L           MLO   200801            24055445  1    4c         1
                         R           MLO   200902            20588046  3    6          1
                                           200901            51049655  2    2          1
                                                             53582710  2    2          1
                                                             53582764  2    2          1
                                                                                      ..
                         L           MLO   201001            50993787  3    2          1
                                                             50994733  3    1          1
                                                             50996056  1    2          1
                                                             50996201  4    2          1
                         R

In [ ]:
# class mapper I-RADS scores of 1 to 3 were mapped as Benign (0), while scores of 4 to 6 were assignedas Malignant (1) as shown

In [ ]:
raw_csv = pd.merge(
    raw_csv,
    raw_csv_[['FileName','Lesion annotation status']],
    how='left',
    on = 'FileName'
)

In [ ]:
data_info = raw_csv.copy()

In [ ]:
ALL_CLASSES = data_info['Lesion annotation status']
ABNORMAL_CLASSES = ALL_CLASSES !='NO ANNOTATION (NORMAL)'
abnormal_images = data_info[ABNORMAL_CLASSES]
data_info.loc[ABNORMAL_CLASSES, ["NEW_CLASS"]] = ["ABNORM"] * abnormal_images.shape[0]
data_info["NEW_CLASS"] = data_info["NEW_CLASS"].fillna("NORM")
data_info = data_info.drop(columns=['Lesion annotation status'])

In [ ]:
IMAGE_DIR = "/kaggle/working/AllPNGs/"
SAVE_DIR = "inbreast"
class_list = ['NORM', 'ABNORM']
for class_name in class_list:
  final_dir = os.path.join(SAVE_DIR, class_name)
  os.makedirs(final_dir, exist_ok=True)


for refnum, class_name in data_info.loc[:, ['FileName','NEW_CLASS']].values:
    in_filename = f"{refnum}.png"
    out_filename = f"{refnum}.png"
   
    img_png = Image.open(os.path.join(IMAGE_DIR, in_filename)).convert("RGB")
    final_dir = os.path.join(SAVE_DIR, class_name)

    img_png.save(os.path.join(final_dir, out_filename))
    # shutil.copyfile(os.path.join(IMAGE_DIR, filename),
    #                 os.path.join(final_dir, out_filename))


In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation((0, 1)),
])

In [ ]:
directory = "/kaggle/working/inbreast"
for dir in os.listdir(directory):
  new_path = os.path.join(directory, dir)

  if dir == "ABNORM":
    for example in os.listdir(new_path):
        example_path = os.path.join(new_path, example)
        img_example = Image.open(example_path).convert("RGB")
        augmented_image = data_augmentation(np.asarray(img_example))
        out_filename = "aug_" + example

        cv2.imwrite(os.path.join(new_path, out_filename), augmented_image.numpy())
      #Image.save(, )


In [ ]:
training_data, validation_data = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=8,
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="both",
    interpolation="bilinear",
)

In [ ]:
training_dataset = training_data.map(pipeline)
validation_dataset = validation_data.map(pipeline)

In [ ]:
tf_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Resultados com freeze=False

In [ ]:
accs = []
for i in range(5):
  model = init_model(freeze=False)
  model.compile(
    loss=tf.keras.losses.binary_crossentropy ,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=['acc'],

  )
  history = model.fit(
      training_dataset,
      epochs=5,
      verbose = 1,
  )
  result = evaluate(model, validation_dataset)
  accs.append(result)
  print("Accuracy: ", result)

In [ ]:
np.mean(accs).round(6), np.std(accs).round(6), np.median(accs).round(6)

In [ ]:
np.std(accs)

# Resultados com freeze=True

In [ ]:
accs_freeze = []
for i in range(5):
  model = init_model(freeze=True)
  model.compile(
    loss=tf.keras.losses.binary_crossentropy ,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=['acc'],

  )
  history = model.fit(
      training_dataset ,
      epochs=5,
      verbose = 1,
  )
  result = evaluate(model, validation_dataset)
  accs_freeze.append(result)
  print("Accuracy: ", result)

In [ ]:
np.mean(accs_freeze).round(3), np.std(accs_freeze).round(3), np.median(accs_freeze).round(3)

In [ ]:
np.std(accs_freeze)